# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-10-10 to 2021-10-07
You have 436 stocks
['A', 'AAL', 'ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AMGN', 'AMP', 'AMT', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AWK', 'AXP', 'AZO', 'BA', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLL', 'BMY', 'BR', 'BRO', 'BSX', 'BWA', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DRE', 'DRI', 'DTE', 'DUK

ticker,A,AAL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01 00:00:00+00:00,155.99,21.65,109.09,119.81,117.21,324.86,577.47,167.37,60.80,202.00,...,35.27,87.21,62.49,60.93,59.07,121.30,123.64,149.47,510.98,196.19
2021-10-04 00:00:00+00:00,152.36,21.39,108.73,118.33,115.53,319.49,558.49,164.26,61.16,199.97,...,35.72,83.56,63.90,61.72,57.63,119.30,123.04,146.73,502.32,193.69
2021-10-05 00:00:00+00:00,153.03,21.47,109.51,118.61,115.88,325.12,566.70,165.37,62.28,203.76,...,35.73,85.05,63.30,61.62,57.58,122.47,123.60,146.01,506.89,196.46
2021-10-06 00:00:00+00:00,153.65,20.54,109.32,116.90,116.49,325.19,570.31,166.05,62.30,204.99,...,35.82,84.25,64.33,60.49,57.46,122.25,123.66,146.34,506.39,195.78
2021-10-07 00:00:00+00:00,155.32,20.17,110.87,118.57,117.91,326.25,578.96,166.98,63.02,207.57,...,35.98,85.49,63.97,60.66,58.00,122.36,125.22,147.74,504.41,199.01


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAL'].tail()

,,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
date,ticker,,,
2021-10-01 00:00:00+00:00,AAL,1.680412,-1.013014,1.378494
2021-10-04 00:00:00+00:00,AAL,1.672466,-0.790548,1.299042
2021-10-05 00:00:00+00:00,AAL,1.545343,-0.178767,1.235480
2021-10-06 00:00:00+00:00,AAL,1.497672,0.941507,0.989178
2021-10-07 00:00:00+00:00,AAL,1.267261,1.537398,0.870000


In [6]:
all_factors.describe()

,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
count,5.314840e+05,5.314840e+05,5.314840e+05
mean,-1.804820e-19,5.213923e-19,-4.177823e-19
std,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.728083e+00,-1.728118e+00,-1.728083e+00
25%,-8.660277e-01,-8.700003e-01,-8.640465e-01
50%,0.000000e+00,-3.972604e-03,0.000000e+00
75%,8.660277e-01,8.620664e-01,8.640465e-01
max,1.728083e+00,1.728118e+00,1.728097e+00


In [7]:
all_factors.head()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2016-12-05 00:00:00+00:00 A                                -0.226438   
                          AAL                              -0.746850   
                          ABBV                             -1.338768   
                          ABC                              -0.997124   
                          ABT                              -0.313836   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2016-12-05 00:00:00+00:00 A                         -0.949460   
                          AAL                        0.583977   
                          ABBV                      -0.981241   
                          ABC                       -0.480689   
                          ABT                        0.242331   

                                  annualzed_volatility_20_day  
date                      ticker                               
2016-12-05 00:00:00+00:00 A                         -1.362614  
                          AAL                       -0.440963  
                          ABBV                       0.591923  
                          ABC                        1.426176  
                          ABT                       -1.386450

In [8]:
all_factors.tail()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-10-07 00:00:00+00:00 XYL                              -0.893836   
                          YUM                              -0.091370   
                          ZBH                              -1.116302   
                          ZBRA                             -0.822329   
                          ZTS                              -0.035753   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2021-10-07 00:00:00+00:00 XYL                        1.426165   
                          YUM                       -0.019863   
                          ZBH                        0.679315   
                          ZBRA                       1.569179   
                          ZTS                       -0.059589   

                                  annualzed_volatility_20_day  
date                      ticker                               
2021-10-07 00:00:00+00:00 XYL                        1.187809  
                          YUM                       -1.648631  
                          ZBH                       -1.418220  
                          ZBRA                       1.450000  
                          ZTS                       -0.027808